In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from statistics import median, mean
from matplotlib import pyplot as plt
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os
import sys
sys.path.insert(0,"/study/mrphys/skunkworks/kk/mriUnet")
from fullmodel3d import fullModel
from unet import UNet
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.model_selection import KFold as kf
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from mriDataset import mriSliceDataset
from smoothing import GaussianSmoothing

In [2]:
b = 1
nchans = 10
f = 16
s = 128
device = 7

model = fullModel().to(device)
x = (torch.randn(b,10,s,s,s) + torch.randn(b,10,s,s,s)*1J).to(device)
gt = (torch.randn(b,10,s,s,s) + torch.randn(b,10,s,s,s)*1J).to(device)
y = (torch.randn(b,1,s,s,s)).to(device)
mask = (torch.randn(b,1,s,s,s)).to(device)

predictions = model(x, gt, y, mask)

In [2]:
b = 1
nchans = 1
f = 16
s = 256
device = 5

model = UNet(
    nchans,
    nchans,
    f_maps=f,
    layer_order=['separable convolution', 'relu'],
    depth=4,
    layer_growth=2.0,
    residual=True,
    complex_input=False,
    complex_kernel=False,
    ndims=3,
    padding=1
)

data = torch.randn(b,nchans,s,s,s) 

model = model.to(device)
data = data.to(device)

predictions = model(data)

In [6]:
mask = torch.randn(b,nchans,s,s,s).to(device)
loss_fn = nn.MSELoss()
loss = loss_fn(predictions*mask, data*mask)

In [8]:
loss.backward()

In [9]:
!nvidia-smi

Mon Jun 19 14:29:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                 Off| 00000000:01:00.0 Off |                  Off |
| 33%   35C    P8               21W / 260W|   1411MiB / 49152MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--